In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L2hvbWUva2V2aW5zaWxiZXJiZXJnL1VDU0MvQklPRTIxNS9kb2NzL3dlZWsy"
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using GLMakie

function makecirc(N::Integer)
    A = rand(N, 2) .- 0.5
    return A ./ sqrt.(sum(A .^ 2, dims = 2))
end

function makefig()
    fig = Figure();
    ax = Axis(fig[1,1], aspect = DataAspect())
    N = 100
    A = Observable(makecirc(N))
    
    points = scatter!([Point2f(A[][i, 1], A[][i, 2]) for i in 1:N])
    on(A) do mat
        points[1] = [Point2f(mat[i, 1], mat[i, 2]) for i in 1:N]
    end
    
    xlims!(ax, -2, 2)
    ylims!(ax, -2, 2)
    return fig, ax, A
end
fig, ax, A = makefig()
save("parta.png", fig)

In [3]:
function getShear(k::Float64)
    return [[1 k]; [k 1]]
end

function getRotation(θ::Float64)
    return [[cos(θ); -sin(θ)] [sin(θ); cos(θ)]]
end

function getStretch(s::Float64)
    return [[s 0]; [0 s]]
end;

In [4]:
shear = getShear(0.01)
record(fig, "shear.mp4", 1:200, framerate = 60) do i
    A[] = A[] * shear
end;

In [5]:
fig, ax, A = makefig()
rotation = getRotation(0.05)
record(fig, "rotation.mp4", 1:200, framerate = 60) do i
    A[] = A[] * rotation
end;

In [6]:
fig, ax, A = makefig()
stretch = getStretch(1.005)
record(fig, "stretch.mp4", 1:200, framerate = 60) do i
    A[] = A[] * stretch
end;

In [7]:
function partD()
    fig = Figure();
    ax1 = Axis(fig[1,1], title = "shear", aspect = DataAspect())
    ax2 = Axis(fig[1,2], title = "rotate", aspect = DataAspect())
    ax3 = Axis(fig[1,3], title = "stretch", aspect = DataAspect())
    xlims!(ax1, -2, 2)
    ylims!(ax1, -2, 2)
    xlims!(ax2, -2, 2)
    ylims!(ax2, -2, 2)
    xlims!(ax3, -2, 2)
    ylims!(ax3, -2, 2)
    
    N = 100
    A = Observable(randn(N, 2))
    B = Observable(randn(N, 2))
    C = Observable(randn(N, 2))
    points1 = scatter!(ax1, [Point2f(A[][i, 1], A[][i, 2]) for i in 1:N])
    points2 = scatter!(ax2, [Point2f(B[][i, 1], B[][i, 2]) for i in 1:N])
    points3 = scatter!(ax3, [Point2f(C[][i, 1], C[][i, 2]) for i in 1:N])    
    on(A) do mat
        points1[1] = [Point2f(mat[i, 1], mat[i, 2]) for i in 1:N]
    end
    on(B) do mat
        points2[1] = [Point2f(mat[i, 1], mat[i, 2]) for i in 1:N]
    end
    on(C) do mat
        points3[1] = [Point2f(mat[i, 1], mat[i, 2]) for i in 1:N]
    end

    record(fig, "partd.mp4", 1:200, framerate = 60) do i
        A[] = A[] * shear
        B[] = B[] * rotation
        C[] = C[] * stretch
    end;
end
partD();

In [8]:
function system(p::Vector{Float64}, N::Integer)
    u = Matrix{Float64}(undef, N, 2)
    u[1, :] = [p[1], p[2]]
    for t = 2:N
        u[t, :] = [0.1*u[t-1, 1] + u[t-1, 2], 0.3*u[t-1, 1] + 0.7*u[t-1, 2]]
    end
    u
end

function makefig()
    fig = Figure();
    ax1 = Axis(fig[1,1], title = "10 juveniles", ylabel = "population", xlabel = "steps")
    xlims!(ax1, 0, 11)
    ylims!(ax1, 0, 20)
    
    p1 = [10.0, 0.0]
    p2 = [0.0, 10.0]
    p3 = [7.0, 7.0]

    sol1 = system(p1, 10)
    sol2 = system(p2, 10)
    sol3 = system(p3, 10)

    lines!(ax1, sol1[:, 1]+sol1[:, 2], label = "10 juveniles", color = :blue)
    lines!(ax1, sol2[:, 1]+sol2[:, 2], label = "10 adults", color = :red)
    lines!(ax1, sol3[:, 1]+sol3[:, 2], label = "7 of each", color = :green)

    Legend(fig[1, 2], ax1, "Legend")

    save("p1partb.png", fig)
end
makefig();

In [9]:
using LinearAlgebra

function partc()
    p1 = normalize([10.0 0.0])
    p2 = normalize([0.0 10.0])
    p3 = normalize([7.0 7.0])

    
    A = [[0.1 1.0];[0.3 0.7]]
    η = eigen(A)

    fig = Figure()
    ax = Axis(fig[1, 1], title = "λ₁ = $(round(η.values[1], digits = 3)), λ₂ = $(round(η.values[2], digits = 3))")
    arrows!(ax, [0.0], [0.0], p1[:, 1], p1[:, 2], color = :blue, label = "10 juveniles")
    arrows!(ax, [0.0], [0.0], p2[:, 1], p2[:, 2], color = :red, label = "10 adults")
    arrows!(ax, [0.0], [0.0], p3[:, 1], p3[:, 2], color = :green, label = "7 of each")
    arrows!(ax, [0.0], [0.0], [η.vectors[1, 1]], [η.vectors[2, 1]], color = :purple, label = "η₁")
    arrows!(ax, [0.0], [0.0], [η.vectors[1, 2]], [η.vectors[2, 2]], color = :orange, label = "η₂")
    Legend(fig[1, 2], ax, "Legend")    
    save("q2partc.png", fig)
    return η, [10.0 0.0], [0.0 10.0], [7.0 7.0]
end
η, p1, p2, p3 = partc();

In [10]:
function project(v::Vector{Float64}, u::Vector{Float64})
    dot(normalize(u), v) * normalize(u)
end

function partd(η, p1, p2, p3)
    ics = [p1, p2, p3]
    iclabels = ["J", "A", "B"]
    eiglabels = ["η₁", "η₂"]

    projections = [project(p, η.vectors[:, i]) for i in 1:2, p in ics]

    fig = Figure()
    ax = Axis(fig[1, 1],
              title = "projections of J, A, B onto η₁, η₂",
              xticks = (1:6, vec([ic * "-" * eig for eig in eiglabels, ic in iclabels])))

    barplot!(ax, vec(projections))
end

partd (generic function with 1 method)